In [1]:
import os 
import sys
from tqdm import tqdm
import importlib
import numpy as np
import pickle
import tensorflow as tf
import matplotlib.pyplot as plt

module_path = "/home/lun/project-basileus/multitype-sequence-generation-by-tlstm-gan/"
data_path = "/home/lun/project-basileus/multitype-sequence-generation-by-tlstm-gan/data/long_seqs_v10/"

if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
tf.__version__

'2.1.0'

In [4]:
# if 'sgtlstm' in sys.modules:
#     importlib.reload(sys.modules['sgtlstm'])

# from sgtlstm.utils import create_dataset, recover_timedelta_to_timestamp
# from sgtlstm.SeqGan import build_G, build_D, build_critic
# from sgtlstm.oracle import get_G_metrics, get_hidden_metrics
# from sgtlstm.TimeLSTM import TimeLSTM0, TimeLSTM1, TimeLSTM2, TimeLSTM3
# from sgtlstm.train import train_discriminator, train_generator, generate_sequences

## Load Data

In [ ]:
pos_data_path = os.path.join(data_path, 'positive_long_sequences.pickle')
neg_data_path = os.path.join(data_path, 'negative_long_sequences.pickle')

def load_sequence_from_pickle_to_numpy(pickle_file_path):
    """
        A list of sequence in format of (event_type, time_delta)
    :param pickle_file_path: e.g. /.../project-basileus/seq-gan/data/fixed_length/valid_sequences.pickle
    :return: (event_type_seqs, time_delta)
    """
    with open(pickle_file_path, 'rb') as f:
        raw_seqs = pickle.load(f)

    if not raw_seqs or not raw_seqs[0]:
        return np.array([]), np.array([])

    N = len(raw_seqs)
    T = len(raw_seqs[0])
    
    seqs = np.array(raw_seqs)
#     print(seqs.shape)
    
    et_seqs = seqs[:, :, 0].astype(np.float64).reshape((N, T, 1))
    ts_seqs = seqs[:, :, 1].astype(np.float64).reshape((N, T, 1))
    return et_seqs, ts_seqs
    
pos_event_type_seqs, pos_timestamp_seqs = load_sequence_from_pickle_to_numpy(pos_data_path)
neg_event_type_seqs, neg_timestamp_seqs = load_sequence_from_pickle_to_numpy(neg_data_path)

In [5]:
BATCH_SIZE = 64
T = 20 + 1
VOCAB = ['END/PADDING', 'INIT', 'start', 'view', 'click', 'install']
EVENT_VOCAB_DIM = len(VOCAB)
EMB_DIM = 32
HIDDEN_DIM = 32

END_TOKEN = 0
MAX_TIME = 1024

## Create original SeqGan

In [70]:
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
from tensorflow.keras import Sequential
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.layers import Input, LSTM, Embedding, Reshape, Dense, Dropout, Conv1D, Conv2D, MaxPooling1D, Activation, Multiply, Add, Lambda
from tensorflow.keras import regularizers

# from sgtlstm.TimeLSTM import TimeLSTM0, TimeLSTM1, TimeLSTM2, TimeLSTM3

tf.keras.backend.set_floatx('float64')


def build_G(batch_size, event_vocab_dim, emb_dim, hidden_dim=11):
    # normal LSTM
    i_et = Input(batch_shape=(batch_size, None, 1), name='event_type')  # input of discrete feature event type
    mask_layer = tf.keras.layers.Masking(mask_value=0., input_shape=(None, 1))
    masked_et = mask_layer(i_et)
    
    embed0 = Embedding(input_dim=event_vocab_dim, output_dim=emb_dim, mask_zero=True)(masked_et)
    # turn [batch_size, None, 1, emb_dim] -> [batch_size, None, emb_dim]
    embed0 = tf.squeeze(embed0, axis=2)

    hm = LSTM(hidden_dim, 
              name='lstm',
              stateful=True,
              return_sequences=False,
              kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.1),
              recurrent_initializer=tf.keras.initializers.RandomNormal(stddev=0.1),
              bias_initializer=tf.keras.initializers.RandomNormal(stddev=0.1))(embed0)

    logits = Dense(event_vocab_dim,
                   activation='linear',
                   name='dense_1',
                   kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.1),
                   bias_initializer=tf.keras.initializers.RandomNormal(stddev=0.1))(hm)
    
    generator = Model(
        inputs=i_et,
        outputs=logits)
        
    return generator

In [40]:
# Modified from: https://gist.github.com/iskandr/a874e4cf358697037d14a17020304535
def highway_layers(value, n_layers, activation="tanh", gate_bias=-3):
    dim = K.int_shape(value)[-1]
    gate_bias_initializer = tf.keras.initializers.Constant(gate_bias)
    for i in range(n_layers):     
        gate = Dense(units=dim, bias_initializer=gate_bias_initializer, activation='sigmoid')(value)
        negated_gate = Lambda(
            lambda x: 1.0 - x,
            output_shape=(dim,))(gate)
        transformed = Dense(units=dim, activation=activation)(value)
        transformed_gated = Multiply()([gate, transformed])
        identity_gated = Multiply()([negated_gate, value])
        value = Add()([transformed_gated, identity_gated])
    return value

In [88]:
def build_D(T, event_vocab_dim, emb_dim, num_filters=100, kernel_size=4, dropout_rate=0.25):
    # Highway network
    i_et = Input(shape=(T, 1), name='event_type')  # input of discrete feature event type

    mask_layer = tf.keras.layers.Masking(mask_value=0., input_shape=(T, 1))
    masked_et = mask_layer(i_et)
    
    embed0 = Embedding(input_dim=event_vocab_dim, output_dim=emb_dim, mask_zero=True)(masked_et)
    embed0 = tf.squeeze(embed0, axis=2)

    conv1 = Conv1D(
        filters=num_filters,
        kernel_size=kernel_size,
        strides=1,
        padding="valid",
        activation='relu',
        use_bias=True,
        kernel_initializer=tf.keras.initializers.TruncatedNormal(mean=0.0, stddev=0.1, seed=None),
        bias_initializer=tf.keras.initializers.Constant(value=0.1))(embed0)
    
    pooled = MaxPooling1D(pool_size=T-kernel_size+1, strides=1)(conv1)
    
    highwayed = highway_layers(pooled, n_layers=1, activation="relu", gate_bias=0)

    dropped = Dropout(rate=dropout_rate)(highwayed)
    
    prob = Dense(1, 
                 activation='sigmoid',
                 name='final',
                 kernel_initializer=tf.keras.initializers.TruncatedNormal(mean=0.0, stddev=0.1, seed=None),
                 bias_initializer=tf.keras.initializers.Constant(value=0.1))(dropped)

    discriminator = Model(
        inputs=i_et,
        outputs=prob)

    return discriminator

## Test initial G and D

In [76]:
G = build_G(batch_size=BATCH_SIZE, event_vocab_dim=11, emb_dim=EMB_DIM, hidden_dim=HIDDEN_DIM)

In [77]:
seq_batch_1 = tf.cast(tf.random.uniform([BATCH_SIZE, T-1, 1], maxval=10, dtype=tf.int32)+1, tf.float32)
# seq_batch_1

In [78]:
G(seq_batch_1)


If you intended to run this layer in float64, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float32 by default, call `tf.keras.backend.set_floatx('float32')`. To change just this layer, pass dtype='float32' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



<tf.Tensor: shape=(64, 11), dtype=float64, numpy=
array([[ 1.24532494e-01,  1.35977862e-01,  1.29068545e-02,
         8.69611896e-02,  6.68012479e-03,  1.26421437e-01,
         1.34822924e-01,  1.92397021e-01,  2.74214513e-02,
        -3.32622989e-02,  1.23282021e-01],
       [ 1.27938958e-01,  1.39523936e-01,  8.06444316e-03,
         8.73132203e-02,  1.60429151e-03,  1.36585068e-01,
         1.28656737e-01,  1.93384279e-01,  3.37601991e-02,
        -3.03049835e-02,  1.21878923e-01],
       [ 1.28938316e-01,  1.38088202e-01,  8.21128069e-03,
         8.62106733e-02,  4.55541098e-03,  1.34817491e-01,
         1.29749337e-01,  1.96272045e-01,  3.38650487e-02,
        -3.12772715e-02,  1.19194812e-01],
       [ 1.26550571e-01,  1.32910429e-01,  1.46359564e-02,
         8.65470458e-02,  9.74773411e-03,  1.25543918e-01,
         1.33351715e-01,  1.95505562e-01,  3.17771794e-02,
        -2.74189690e-02,  1.27151926e-01],
       [ 1.30178825e-01,  1.35456911e-01,  1.51173010e-02,
         8.

In [89]:
D = build_D(T=20, event_vocab_dim=11, emb_dim=32, num_filters=100, kernel_size=4, dropout_rate=0.25)

In [90]:
D(seq_batch_1)


If you intended to run this layer in float64, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float32 by default, call `tf.keras.backend.set_floatx('float32')`. To change just this layer, pass dtype='float32' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



<tf.Tensor: shape=(64, 1, 1), dtype=float64, numpy=
array([[[0.48348546]],

       [[0.48213291]],

       [[0.48017302]],

       [[0.48463881]],

       [[0.48266518]],

       [[0.48393663]],

       [[0.48243204]],

       [[0.48065238]],

       [[0.4797539 ]],

       [[0.4816363 ]],

       [[0.48112918]],

       [[0.48347778]],

       [[0.48292298]],

       [[0.48146941]],

       [[0.48264483]],

       [[0.48504926]],

       [[0.48501926]],

       [[0.48131647]],

       [[0.48118718]],

       [[0.48470553]],

       [[0.48090029]],

       [[0.48062   ]],

       [[0.48344876]],

       [[0.47887864]],

       [[0.48134473]],

       [[0.47979953]],

       [[0.48142653]],

       [[0.48891772]],

       [[0.47813068]],

       [[0.48147723]],

       [[0.48065958]],

       [[0.48127611]],

       [[0.48318802]],

       [[0.48179609]],

       [[0.483459  ]],

       [[0.48380761]],

       [[0.48272003]],

       [[0.4807248 ]],

       [[0.48305384]],

       [[0.4